# Human Heat Metrics

Wet Bulb Glove Temperature ($WBGT$) is a measure of heat stress outdoors ($WBGT_{OD}$) or indoors/in the shade ($WBGT_{ID}$).

Dry Bulb  Ambient Temperatue ($T_a$) - what you’d think of as “just temperature”, or ambient temperature. Does not account for radiation or humidity.

Thermodynamic Wet Bulb Temperature ($T_{pwb}$) wet bulb temperature in the shade and fanned or rotated. This is the wet bulb typically used for dew point calculations.

Natural Wet Bulb Temperature ($T_{nwb}$) wet bulb temperature with exposure to wind and sun. This is not a readily accessible measurement.

Globe Temperature ($T_g$) temperature taken from inside a copper globe painted black and exposed to the sun. Also not a readily accessible measurement.


$WBGT_{od} = 0.7*T_{nwb} + 0.2*T_g + 0.1*T_a$

$WBGT_{id} = 0.7*T_{nwb} + 0.3*T_g$

In [2]:
import math
import scipy

## Liljergen (Argonne Model)

This is a method of approximating Globe Temperature ($T-g$).

This model:

- assumes that when there is no solar radiation, 
- assumes $T_`{pwb}$ = $T_{nwb}$
- and tends to underestimate indoor $T_{nwb}$

In [ ]:

f_dir = ' ' # fraction due to diffuse solar irradiance
S = ' ' # total horizontal solar irradiance


def _get_e_sat(T): # Magnus-Tetens Approximation
    e_sat = 0.61094 * math.exp((17.625 * T) / (T + 243.04)) # Saturation Vapor Pressure in kPa
    return e_sat


def _get_theta(latitude, longitude, datetime):
    phi = latitude

    time = ' ' # from datetime
    day_of_year = ' ' # Day of Year from datetime
    
    delta = 23.45  * math.sin(math.radians(360/365 * (284 + day_of_year))) # solar declination

    time_correction = ' ' # from longitude and day_of_year
    solar_time = ' ' # from time and time_correction
    hour_angle = 15 * (12 - solar_time) # degrees

    cos_theta = (
        (math.sin(phi) * math.sin(delta))
        + (math.cos(phi) * math.cos(delta) * math.cos(math.radians(hour_angle)))
        )
    theta = math.acos(cos_theta)
    return theta


def calc_liljergen_wet_bulb(T_a, # Ambient Temperature)
                            rh, # Relative Humidity
                            h, # Convective Heat Transfer Coefficient
                            datetime, # Time of Day
                        ):
    epsilon_g = 0.95 # Globe emissivity
    alpha_sfc = 0.45 # Surface albedo
    sigma = scipy.constants.Stefan_Boltzmann # Stefan-Boltzmann Constant

    e_sat = _get_e_sat(T_a) # Saturation Vapor Pressure
    epsilon_a = 0.575 * (rh * e_sat(T_a)) ** 0.143 # Atmosphere Emissivity

    theta = _get_theta(datetime) # Solar Zenith Angle

    # Convergence parameters
    tolerance = 1e-6
    max_iterations = 100

    # Iterative relaxation solve
    T_g = T_a
    for i in range(max_iterations):

        T_g_new = (
            0.5 * (1 + epsilon_a) * T_a ** 4
            - h / (epsilon_g * sigma) * (T_g - T_a)
            + S / (2 * epsilon_g * sigma) * (1 + f_dir * (1 / (2 * math.cos(theta)) - 1) + alpha_sfc)
        ) ** 0.25
        
        # Check for convergence
        if abs(T_g_new - T_g) < tolerance:
            T_g = T_g_new
            break
        
        T_g = T_g_new
    return T_g


def calc_liljergen_wbgt(T_a, T_nwb, *kwargs):
    T_g = calc_liljergen_wet_bulb(*kwargs)
    WBGT = 0.7*T_nwb + 0.2*T_g + 0.1*T_a
    return WBGT

## Bernard

This is a method of estimating indoor Wet Bulb Globe Temperature ($WBGT_{id}$) that does not consider sunny conditions.

In [ ]:
def calc_bernard_wbgt(t_a, t_pwb, v):
    if v < 0.3: #m/s
        return
    elif 0.3 <= v <= 3:
        wbgt = (0.67 * t_pwb) + (0.33 * t_a) - (0.48 * math.log10(v) * (t_a - t_pwb))
    else:
        wbgt = (0.7 * t_pwb) + (0.3 * t_a)
    return wbgt

## Australian Bureau of Meteorology (ABM)

Meethod of estimating Wet Bulb Global Temperature

- tends to overpredict WBGT
- assumes full sunlight and light breeze

In [ ]:
def calc_abm_wbgt(t_a, rh):
    e = (rh / 100)**17.27 * (t_a/(337.7 * t_a)) # water vapor pressure [hPa]
    wbgt = (0.567 * t_a) + (0.393 * e) + 3.94
    return wbgt

## Li

Not much information here